In [ ]:
%pip install --user --quiet ciw
import ciw

## Simulation

In [ ]:
from dataclasses import dataclass, asdict
from typing import List, ClassVar
from tqdm import tqdm

@dataclass
class Distribution:
    kind: str
    mean: float
    d: ClassVar[ciw.dists.Distribution]

@dataclass
class Model:
    num_servers: int
    queue_capacity: int
    arrival_dist: Distribution
    service_dist: Distribution

@dataclass
class SimResult:
    model: ClassVar[Model]
    task_count: int
    utilization: float
    loss_probability: float
    mean_wait_time: float
    mean_residence_time: float
    
    def __init__(self, model: Model, sim: ciw.Simulation):
        self.model = model
        self.utilization = sim.transitive_nodes[0].server_utilisation
        
        tasks = sim.get_all_records()
        self.task_count = len(tasks)
        self.loss_probability = len(sim.rejection_dict[1][0]) / self.task_count
        self.mean_wait_time = sum(t.waiting_time for t in tasks) / self.task_count
        self.mean_residence_time = sum(t.waiting_time + t.service_time for t in tasks) / self.task_count

def simulate(model: Model, task_measures: List[int]) -> List[SimResult]:
    sim = ciw.Simulation(ciw.create_network(
        arrival_distributions=[model.arrival_dist.d],
        service_distributions=[model.service_dist.d],
        number_of_servers=[model.num_servers],
        queue_capacities=[model.queue_capacity]
    ))
    results = []
    for task_count in tqdm(task_measures):
        sim.simulate_until_max_customers(task_count, method='Finish')
        results.append(SimResult(model, sim))
    return results

## Distributions

In [ ]:
@dataclass
class DExp(Distribution):
    def __init__(self, rate):
        self.kind = 'Exponential'
        self.mean = 1 / rate
        self.d = ciw.dists.Exponential(rate)

@dataclass
class DHypoexp2(Distribution):
    def __init__(self, rate1, rate2):
        self.kind = 'Hypoexponential'
        self.mean = 1 / rate1 + 1 / rate2
        self.d = ciw.dists.Exponential(rate1) + ciw.dists.Exponential(rate2)

@dataclass
class DTrace(Distribution):
    def __init__(self):
        with open('trace.txt', 'r') as f:
            trace = [float(v) for v in f.read().splitlines()]

        self.kind = 'Trace'
        self.mean = sum(trace) / len(trace)
        self.d = ciw.dists.Sequential(trace)

## Model Variations

In [ ]:
models = [
    Model(num_servers=2, queue_capacity=10, arrival_dist=DExp(1 / 80), service_dist=DExp(1 / 18)),
    Model(num_servers=2, queue_capacity=10, arrival_dist=DExp(1 / 18), service_dist=DExp(1 / 18)),
    Model(num_servers=2, queue_capacity=10, arrival_dist=DExp(1 / 9), service_dist=DExp(1 / 18))
]

## Data Export

Further analysis is performed in R, see `Report.Rmd`.

In [ ]:
from multiprocessing import Pool
import json

def compute_model(model_def):
    m_idx, model = model_def
    task_counts = [300] + list(range(10_000, 100_000, 5_000)) + list(range(100_000, 400_000, 100_000))
    
    print(f'Running model #{m_idx}')
    results = [asdict(result) for result in simulate(model, task_counts)]
    with open(f'model{m_idx}.json', 'w') as f:
        json.dump(results, f)
    print(f'Saved {len(results)} results for model #{m_idx}')

with open('models.json', 'w') as f:
    json.dump(list(map(asdict, models)), f)

with Pool(8) as pool:
    pool.map(compute_model, enumerate(models))
pool.join()